In [ ]:
import json
import redis
import paho.mqtt.client as mqtt 

#connect to a broker to send/receive messages to/from the broker
client = mqtt.Client()

seriesName = "StutteringEvent"

REDIS_HOST = "redis-11915.c98.us-east-1-4.ec2.cloud.redislabs.com"
REDIS_PORT = 11915
REDIS_USER = "default"
REDIS_PASSWORD = "kn1Ktpz2ObSEwX7s1lA9KttX9IYgy5yC"

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [ ]:
def check_series_creation(redis_client, sensor):
    '''checks for the existence of the series and creates them'''
    try:
        redis_client.ts().create(sensor+f":{seriesName}", label = {f"{seriesName}"})
    except redis.ResponseError:
        pass
    

def on_connect(client, userdata, flags, rc):
    #subscribe to a topic
    client.subscribe('s296721')


def on_message(client, userdata, msg):
    message = msg.payload.decode() 
    message_dict = json.loads(message)
    print(message)
    #whenever a new sensor is received, we need to create the redis timeseries
    check_series_creation(redis_client, message_dict["sensor"])
    
    #store the data
    redis_client.ts().add(message_dict["sensor"]+f":{seriesName}", message_dict["timestamp"], message_dict["stutteringEvent"])
    


#bind the on_connect and on_message method to the client
client.on_connect = on_connect
client.on_message = on_message

#connect to the broker 
client.connect('mqtt.eclipseprojects.io', 1883)

0

In [0]:
#keep listening over the topic we subscribed to using an infinite loop 
client.loop_forever()

{"sensor": "Martini", "timestamp": 1688653771766, "stutteringEvent": 1}
{"sensor": "AmedeoDiSavoia", "timestamp": 1688653789815, "stutteringEvent": 1}
{"sensor": "ClinicaPinnaPintor", "timestamp": 1688653791758, "stutteringEvent": 1}
{"sensor": "FornacaClinic", "timestamp": 1688653805306, "stutteringEvent": 1}
{"sensor": "Martini", "timestamp": 1688653806700, "stutteringEvent": 1}
{"sensor": "Koelliker", "timestamp": 1688653811370, "stutteringEvent": 1}
{"sensor": "Martini", "timestamp": 1688653814391, "stutteringEvent": 1}
{"sensor": "ClinicaPinnaPintor", "timestamp": 1688653817851, "stutteringEvent": 1}
{"sensor": "FornacaClinic", "timestamp": 1688653819281, "stutteringEvent": 1}
{"sensor": "AmedeoDiSavoia", "timestamp": 1688653823866, "stutteringEvent": 1}
{"sensor": "ClinicaPinnaPintor", "timestamp": 1688653826382, "stutteringEvent": 1}
{"sensor": "Martini", "timestamp": 1688660449549, "stutteringEvent": 1}
{"sensor": "Martini", "timestamp": 1688660549792, "stutteringEvent": 1}
{"s

ResponseError: TSDB: Error at upsert, update is not supported when DUPLICATE_POLICY is set to BLOCK mode

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>